In [1]:
LIGHT_IMPORT = True

%run -i "Calc_Embeddings_Model Training.ipynb"

# quick sanity checks
print("df shape:", df.shape if 'df' in globals() else 'df missing')
print("have get_matches?", 'get_matches' in globals())

shape: (59946, 31)


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


QUERY user 1234 | age 31 | m straight | palo alto, california
------------------------------------------------------------------------------------------
hello! thanks for stopping by! i hope you enjoy yourself!  i am extremely financially responsible and nice! need a new kidney? i'll give you mine! if the doctor won't let us do that, i'll just buy you a new one! no expense is too much for you!  i definitely have skinny genes! rea...

=== Matches ===

user 44713 | age 23 | m straight | hayward, california | score=0.594
hey my name is jacob i'm a die hard raider! lol i work as a massage therapist at chiropractic offices and my personal clients i do some personal tranning generally weight loss, nutrition, and hypertrophy. i'm currently attending ohlone college working on a physical therapy assist...
------------------------------------------------------------------------------------------
user 10612 | age 28 | m gay | san francisco, california | score=0.564
high energy goofy nerd-jock-goo

In [2]:
print("have compute_component_scores?", 'compute_component_scores' in globals())

have compute_component_scores? True


In [63]:
# ==== BUMBLE CONFIG ====

# Candidate pool / retrieval
BUMBLE_POOL_K      = 5000    # how many candidates to fetch from get_matches for the user
BUMBLE_MAX_KM      = 20      # geo radius for get_matches
BUMBLE_TOP_K       = 20      # how many recs to return

# Scoring blend (used later with compute_component_scores output)
# We'll combine: text_sim, age_score, lifestyle_score, loc_score
BUMBLE_WEIGHTS = {
    "text": 0.75,   # semantic similarity (or 'score' mapped to text_sim)
    "age":  0.05,   # age compatibility
    "life": 0.15,   # lifestyle alignment (drinks/smokes/drugs/diet if available)
    "loc":  0.05,   # distance score
}

# Messaging behavior
BUMBLE_OPENER_N          = 3       # how many suggested openers to generate per match

# Reproducibility for any sampling we do in this notebook
BUMBLE_RANDOM_SEED = 123

In [64]:
def ensure_text_sim(cands: pd.DataFrame) -> pd.DataFrame:
    c = cands.copy()
    if "text_sim" not in c.columns and "score" in c.columns:
        c["text_sim"] = c["score"]
    c["text_sim"] = c["text_sim"].fillna(0).clip(lower=0)
    return c

In [78]:
def bumble_ranking(user_id: int,k: int = BUMBLE_TOP_K, pool_k: int = BUMBLE_POOL_K,
                      max_km: float = BUMBLE_MAX_KM) -> pd.DataFrame:
    """
    1) Pull candidates with hard gates via get_matches.
    2) compute_component_scores to get text/age/lifestyle/loc scores.
    3) Blend into bumble_score using apply_weights (fixed weights).
    4) Return top-k ranked table.
    """
    # 1) candidate pool
    cands = get_matches(user_id=user_id, k=pool_k,use_orientation=True, use_location=True, max_km=max_km,
        use_age=True, use_diet=False, use_drinks=False, use_smokes=True, use_drugs=True,
        drinks_strict=False, smokes_strict=True, drugs_strict=True, allow_missing=True).copy()

    # 2) component scores
    cands = compute_component_scores(user_id=user_id, cands=cands, max_km=max_km,
        drinks_strict=False, smokes_strict=False, drugs_strict=False, allow_missing=True)

    # 3) blend into final score (fixed weights)
    ranked = apply_weights(cands,w_text=BUMBLE_WEIGHTS["text"],w_age =BUMBLE_WEIGHTS["age"],
        w_life=BUMBLE_WEIGHTS["life"],w_loc =BUMBLE_WEIGHTS["loc"]).copy()
    # rename final_score → bumble_score
    if "final_score" in ranked.columns:
        ranked = ranked.rename(columns={"final_score": "bumble_score"})

    # 4) clean output
    need_cols = ["user_id","bumble_score","text_sim","age_score","lifestyle_score","loc_score",
        "distance_km","location","age","sex","orientation"]
    for col in need_cols:
        if col not in ranked.columns:
            ranked[col] = np.nan

    return (ranked[need_cols].sort_values("bumble_score", ascending=False).head(k).reset_index(drop=True))


In [76]:
u = 2
# run the Bumble ranker
bumble_list = bumble_ranking(user_id=u, k=20, pool_k=BUMBLE_POOL_K,max_km=BUMBLE_MAX_KM)
# look at the results
bumble_list.head(10)

Start pool: 5000 (max_candidates=5000)
After orientation: 1924
Unfiltered: 1924 | After location filter (≤20 km): 1575
After location (≤20 km): 1575
After age rule: 1200


,user_id,bumble_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation
0,13660,0.744935,0.659914,1.0,1.0,1.0,0.000000,"san francisco, california",27,f,straight
1,12588,0.732446,0.643261,1.0,1.0,1.0,0.000000,"san francisco, california",29,f,straight
2,26526,0.722439,0.629919,1.0,1.0,1.0,0.000000,"san francisco, california",27,f,straight
3,9413,0.722431,0.629908,1.0,1.0,1.0,0.000000,"san francisco, california",30,f,straight
4,37289,0.722098,0.629464,1.0,1.0,1.0,0.000000,"san francisco, california",31,f,straight
5,38592,0.718737,0.624982,1.0,1.0,1.0,0.000000,"san francisco, california",33,f,straight
6,49622,0.717033,0.622711,1.0,1.0,1.0,13.300941,"oakland, california",31,f,straight
7,3462,0.715903,0.621204,1.0,1.0,1.0,0.000000,"san francisco, california",38,f,straight
8,16953,0.715187,0.620249,1.0,1.0,1.0,0.000000,"san francisco, california",42,f,straight
9,25872,0.711365,0.615153,1.0,1.0,1.0,0.000000,"san francisco, california",36,f,straight


In [67]:
# ==== ATTACH BIO TEXT TO RANKED ====
def bumble_add_bio(bumble_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge the master df['bio_text'] onto the ranked Bumble table by user_id.
    Returns the same rows/ordering plus a 'bio_text' column.
    """
    if "bio_text" in bumble_df.columns:
        return bumble_df

    out = bumble_df.merge(df[["user_id", "bio_text"]],on="user_id",how="left")
    return out

In [68]:
u = 14 
bumble_basic = bumble_ranking(user_id=u, k=20, pool_k=BUMBLE_POOL_K, max_km=BUMBLE_MAX_KM)
bumble_with_bio = bumble_add_bio(bumble_basic)
bumble_with_bio.head(3)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤20 km): 522
After location (≤20 km): 522
After age rule: 461


,user_id,bumble_score,text_sim,age_score,lifestyle_score,loc_score,distance_km,location,age,sex,orientation,bio_text
0,35672,0.794078,0.725438,1.0,1.0,1.0,13.449287,"oakland, california",25,m,straight,"first things first, my name is kyle (hence, el..."
1,10444,0.792144,0.722858,1.0,1.0,1.0,19.197039,"berkeley, california",22,m,straight,hmmm... well i'm pretty chill and i like to he...
2,38032,0.783253,0.711003,1.0,1.0,1.0,19.197039,"berkeley, california",27,m,straight,"here's some random stuff about me, since i'm t..."


In [37]:
pip install openai

     -------------------------------------- 810.8/810.8 KB 5.1 MB/s eta 0:00:00
     -------------------------------------- 208.8/208.8 KB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\veera\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [69]:
# ==== BUMBLE: pickup lines for women only (API only) ====
from openai import OpenAI
import os
API_KEY = " "
os.environ["OPENAI_API_KEY"] = API_KEY

def truncate_bio(bio: str, max_words: int = 30) -> str:
    """Return the first max_words words of bio (with ellipsis if longer)."""
    words = bio.split()
    if len(words) > max_words:
        return " ".join(words[:max_words]) + " ..."
    return bio

def bumble_print_openers_for_user(user_id: int,k: int = BUMBLE_TOP_K,pool_k: int = BUMBLE_POOL_K,
    max_km: float = BUMBLE_MAX_KM,opener_n: int = 3,model: str = "gpt-4o-mini"):
    """
    If the user's sex is F, generate pickup lines for her top-k matches and print them.
    Uses OpenAI API directly.
    """
    user = df.loc[df["user_id"] == user_id]
    user_sex = str(user["sex"].iloc[0]).strip().upper()
    if user_sex != "F":
        print(f"User {user_id} is sex={user_sex!r} — per Bumble rules, he cannot initiate. Skipping openers.")
        return

    # Build ranked + bio
    ranked = bumble_ranking(user_id=user_id, k=k, pool_k=pool_k, max_km=max_km)
    ranked = bumble_add_bio(ranked)

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    for _, r in ranked.iterrows():
        uid   = r["user_id"]
        score = r["bumble_score"]
        age   = r["age"]
        loc   = r["location"]
        bio   = (r["bio_text"] or "").strip()

        # Call API to generate openers
        prompt = (
            "Write short, playful Bumble openers tailored to the following profile bio. "
            f"Return {opener_n} separate lines. Keep each under 18 words.\n\n"
            f"BIO:\n{bio}\n" )
        resp = client.chat.completions.create(model=model,messages=[{"role":"user","content": prompt}],
            temperature=0.8)
        text = resp.choices[0].message.content.strip()
        openers = [ln.strip() for ln in text.split("\n") if ln.strip()]

        # Print block
        print(f"\nBumble Score = {score:.4f}" if isinstance(score,(int,float)) else f"\nBumble Score = {score}")
        print(f"User id = {uid}      age = {age}      location = {loc}")
        print("bio text =")
        print(truncate_bio(bio, max_words=30))
        print("Suggested pick up lines =")
        for i, line in enumerate(openers, 1):
            print(f" {line}")

In [70]:
u = 42
bumble_print_openers_for_user(user_id=u, k=5)

User 42 is sex='M' — per Bumble rules, he cannot initiate. Skipping openers.


In [71]:
u = 169
bumble_print_openers_for_user(user_id=u, k=1)

Start pool: 5000 (max_candidates=5000)
After orientation: 2401
Unfiltered: 2401 | After location filter (≤20 km): 1863
After location (≤20 km): 1863
After age rule: 1718

Bumble Score = 0.7705
User id = 1540      age = 33      location = oakland, california
bio text =
i'm not good at elevator pitches but here goes: my favorite color is yellow. i think i'm right most of the time but i love being proved wrong. my friends ...
Suggested pick up lines =
 1. If you could only ride your bike to one place forever, where would it be? 🌍🚴‍♀️
 2. Yellow is the color of happiness! What's your happiest memory involving it? 💛✨
 3. What's your secret to making the best soup? I might need to take notes! 🍲🥳


In [74]:
# ==== eHarmony: guided questions from bio (API only) ====
from openai import OpenAI
import os
API_KEY = " "
os.environ["OPENAI_API_KEY"] = API_KEY

def truncate_bio(bio: str, max_words: int = 30) -> str:
    """Return the first max_words words of bio (with ellipsis if longer)."""
    words = bio.split()
    if len(words) > max_words:
        return " ".join(words[:max_words]) + " ..."
    return bio

def eharmony_print_questions_for_user(user_id: int,k: int = BUMBLE_TOP_K,pool_k: int = BUMBLE_POOL_K,
    max_km: float = BUMBLE_MAX_KM,question_n: int = 3,model: str = "gpt-4o-mini"):
    """
    eHarmony-style: show top-k matches and generate thoughtful, relationship-focused questions
    based on each match's bio_text.
    """

    # Build ranked + bio
    ranked = bumble_ranking(user_id=user_id, k=k, pool_k=pool_k, max_km=max_km)
    ranked = bumble_add_bio(ranked)

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    for _, r in ranked.iterrows():
        uid   = r["user_id"]
        score = r["bumble_score"]   
        age   = r["age"]
        loc   = r["location"]
        bio   = (r["bio_text"] or "").strip()

        # Call API to generate openers
        prompt = ("You are helping two people see if they are compatible for a serious relationship. "
            f"Read the profile bio below and write {question_n} short, thoughtful questions that spark meaningful conversation. "
            "Keep each question under 18 words, avoid clichés, and vary themes (values, lifestyle, future plans, hobbies). "
            "Return the questions as separate lines with no numbering.\n\n"
            f"BIO:\n{bio}\n")

        resp = client.chat.completions.create(model=model,messages=[{"role":"user","content": prompt}],temperature=0.7)
        text = resp.choices[0].message.content.strip()
        questions = [ln.strip() for ln in text.split("\n") if ln.strip()]
        
        # Print block
        print(f"\nCompatibility Score = {score:.4f}" if isinstance(score,(int,float)) else f"\nCompatibility Score = {score}")
        print(f"User id = {uid}      age = {age}      location = {loc}")
        print("bio text =")
        print(truncate_bio(bio, max_words=30))
        print("Suggested questions to ask =")
        for q in questions[:question_n]:
            print(f"  {q}")

In [75]:
u = 14  # any user_id you want to get suggestions for
eharmony_print_questions_for_user(user_id=u, k=1, question_n=1)

Start pool: 5000 (max_candidates=5000)
After orientation: 2478
Unfiltered: 2478 | After location filter (≤20 km): 522
After location (≤20 km): 522
After age rule: 461

Compatibility Score = 0.7941
User id = 35672      age = 25      location = oakland, california
bio text =
first things first, my name is kyle (hence, elyk). i was born in the small town of henderson, kentucky and after graduating college in virginia i've found myself out here ...
Suggested questions to ask =
  What creative project are you most excited about right now, and why does it inspire you?


In [79]:
import pandas as pd, json, os
from datetime import datetime

# ensure folder
os.makedirs("results", exist_ok=True)

# run Bumble algorithm
bumble = bumble_ranking(user_id=15, k=100, pool_k=500, max_km=45)

# standardize + annotate
bumble = bumble.reset_index(drop=True).copy()
bumble["rank"] = range(1, len(bumble) + 1)
bumble["app"]  = "bumble"

# metadata relevant to Bumble
meta = {
    "app": "bumble",
    "user_id": 15,
    "top_n": 100,
    "pool_k": 500,
    "max_km": 45,
    "weights": {"text":0.75, "age":0.05, "life":0.15, "loc":0.05},  # from BUMBLE_WEIGHTS
    "timestamp": datetime.utcnow().isoformat() + "Z",
}

# consistent stem (no colons, matches others)
stem = f"results/bumble_user_id:{meta['user_id']}_k{meta['top_n']}_pool{meta['pool_k']}_km{meta['max_km']}"

# save
bumble.to_parquet(f"{stem}.parquet", index=False)
with open(f"{stem}.json","w") as f: json.dump(meta, f, indent=2)

print("Saved:", f"{stem}.parquet and {stem}.json")

Start pool: 5000 (max_candidates=5000)
After orientation: 2005
Unfiltered: 2005 | After location filter (≤45 km): 1941
After location (≤45 km): 1941
After age rule: 1478
Smokes | before: 1478 → after: 1277 (strict=True)
After smokes (strict=True): 1277
Drugs  | before: 1277 → after: 1277 (strict=True)
After drugs (strict=True): 1277
Saved: results/bumble_user_id:15_k100_pool500_km45.parquet and results/bumble_user_id:15_k100_pool500_km45.json
